# Sesión 3 - Cuaderno 4: El Rol de las Restricciones y Conclusiones

### **Introducción: Las "Pistas" para el Optimizador**

A lo largo de esta sesión, hemos visto cómo el optimizador de Spark utiliza estadísticas y la estructura física de los datos (particiones, Z-Ordering) para acelerar las consultas. Sin embargo, hay una última pieza del rompecabezas que le proporciona información invaluable: las **restricciones de integridad referencial** (claves primarias y foráneas).

En la Sesión 2, declaramos estas restricciones con la cláusula `NOT ENFORCED`. Esto significa que Delta Lake **no valida activamente** que se cumplan (no rechaza filas que violen la unicidad de una PK), pero sí **registra la relación en sus metadatos**.

¿Por qué hacemos esto? Porque le damos "pistas" al optimizador Catalyst que puede usar para simplificar los planes de ejecución.

**Objetivo:**
* Entender cómo las restricciones `NOT ENFORCED` ayudan al optimizador.
* Consolidar todo lo aprendido en un ejercicio práctico final.
* Resumir las mejores prácticas para el diseño y mantenimiento de tablas.

### **Paso 1: El Efecto de las Claves Primarias y Foráneas**

Cuando declaramos una relación PK-FK, le estamos haciendo una promesa al optimizador: "Te garantizo que por cada fila en la tabla de hechos (ej. `pedidos_silver`), hay como máximo una fila correspondiente en la tabla de dimensión (ej. `usuarios_silver`)".

Incluso si es `NOT ENFORCED`, Catalyst puede usar esta promesa para realizar optimizaciones como la **eliminación de `JOINs` redundantes**.

**Escenario:**
Imagina una consulta que solo necesita filtrar por un atributo de la dimensión, pero el resultado final no requiere ninguna columna de esa dimensión.



In [0]:
# Celda de configuración
# Establecemos el contexto de la base de datos para la sesión actual.

db_name = "curso_arquitecturas"
spark.sql(f"USE {db_name}")

print(f"Contexto de base de datos activo: '{db_name}'")

In [0]:
%sql
SELECT
  p.monto_total
FROM pedidos_silver p
JOIN usuarios_silver u
  ON p.id_usuario = u.id_usuario
WHERE p.id_usuario = 1025;

In [0]:
%sql
-- Queremos el monto total de los pedidos realizados por el usuario 1025.
-- No necesitamos el nombre del usuario, solo su ID para filtrar.
EXPLAIN SELECT
  p.monto_total
FROM pedidos_silver p
JOIN usuarios_silver u
  ON p.id_usuario = u.id_usuario
WHERE p.id_usuario = 1025;

In [0]:
%sql
SELECT
  p.monto_total
FROM pedidos_silver p
JOIN usuarios_silver u
  ON p.id_usuario = u.id_usuario
WHERE u.id_usuario = 1025;

**Análisis del Plan:**
En un escenario sin restricciones, Spark podría realizar el `JOIN` innecesariamente. Sin embargo, al tener la restricción PK-FK, el optimizador puede razonar: "El `JOIN` con `usuarios_silver` no añade ni elimina filas de `pedidos_silver` debido a la relación 1 a N, y como no se necesita ninguna columna de `usuarios_silver` en el resultado, **puedo eliminar el `JOIN` por completo**".

El plan físico resultante será mucho más simple, mostrando solo un `Scan` sobre `pedidos_silver` con el filtro aplicado, ignorando por completo la tabla `usuarios_silver`.

### **Paso 2: El Ingrediente Secreto - Las Estadísticas de la Tabla**

Todas las decisiones inteligentes que hemos visto tomar a Catalyst (elegir un `BroadcastHashJoin` sobre un `SortMergeJoin`, realizar `file skipping` con Z-Ordering) dependen de un ingrediente crucial: **información precisa y actualizada sobre los datos**.

Aquí es donde entra el comando `ANALYZE TABLE`.

* **¿Qué hace?** `ANALYZE TABLE` escanea la tabla y recolecta metadatos detallados (estadísticas) sobre los datos. Esto incluye:
    * Número total de filas.
    * Tamaño de la tabla en bytes.
    * Para cada columna: valores mínimos y máximos, número de valores nulos, número de valores distintos, histogramas de distribución, etc.
* **¿Por qué es importante?** Estas estadísticas alimentan el **Modelo de Costos** del optimizador. Con estadísticas precisas, Catalyst puede tomar decisiones casi perfectas. Sin ellas, tiene que recurrir a estimaciones, lo que puede llevar a planes de ejecución subóptimos.



In [0]:
%sql
-- Así se recolectan las estadísticas para una tabla
ANALYZE TABLE pedidos_silver COMPUTE STATISTICS FOR ALL COLUMNS


**Regla de Oro:** Ejecuta `ANALYZE TABLE` después de cargas de datos masivas o modificaciones significativas en una tabla para asegurar que el optimizador siempre trabaje con la información más reciente.

In [0]:
%sql DESCRIBE EXTENDED pedidos_silver

### **Paso 3: Resumen de Mejores Prácticas de Optimización**

Aquí tienes una guía de referencia rápida para mantener tus tablas Delta Lake funcionando al máximo rendimiento.

| Práctica | Comando | Cuándo Usarlo | Beneficio Principal |
| :--- | :--- | :--- | :--- |
| **Particionamiento** | `PARTITIONED BY (col)` | Durante la creación de la tabla. | **Directory Skipping**. Ideal para columnas de baja cardinalidad como fechas. |
| **Z-Ordering** | `OPTIMIZE ... ZORDER BY (cols)` | Periódicamente, después de cargas de datos. | **File Skipping**. Ideal para columnas de alta cardinalidad usadas en filtros (`id_usuario`). |
| **Compactación** | `OPTIMIZE table_name` | Periódicamente, si hay muchos archivos pequeños. | Fusiona archivos pequeños en grandes, acelerando las lecturas. |
| **Actualizar Estadísticas** | `ANALYZE TABLE ...` | Después de cargas o modificaciones importantes. | Alimenta al optimizador para que tome las mejores decisiones. |
| **Limpieza** | `VACUUM table_name` | Periódicamente (a menudo automatizado). | Elimina archivos antiguos que ya no son referenciados por el log de transacciones. |

### **Paso 4: Ejercicio Final - Optimizando una Consulta de Negocio**

Es hora de aplicar todo lo que hemos aprendido. El equipo de negocio necesita un reporte de **los 5 productos más devueltos por clientes de la ciudad 'Madrid' durante la primera semana de Febrero de 2023.**

Aquí tienes una primera versión de la consulta:

In [0]:
%sql
-- Versión 1: Consulta sin optimizar
EXPLAIN EXTENDED SELECT
  prod.nombre_producto,
  COUNT(*) AS total_devoluciones
FROM devoluciones_silver AS dev
JOIN pedidos_silver_partitioned_user AS ped
  ON dev.id_pedido = ped.id_pedido
JOIN usuarios_silver AS u
  ON ped.id_usuario = u.id_usuario
JOIN productos_silver AS prod
  ON ped.id_producto = prod.id_producto
WHERE
  u.ciudad = 'Madrid'
GROUP BY
  prod.nombre_producto
ORDER BY
  total_devoluciones DESC
LIMIT 5000;


In [0]:
%sql
SELECT
  prod.nombre_producto,
  COUNT(*) AS total_devoluciones
FROM devoluciones_silver AS dev
JOIN pedidos_silver_partitioned_user AS ped
  ON dev.id_pedido = ped.id_pedido
JOIN usuarios_silver AS u
  ON ped.id_usuario = u.id_usuario
JOIN productos_silver AS prod
  ON ped.id_producto = prod.id_producto
WHERE
  u.ciudad = 'Madrid'
GROUP BY
  prod.nombre_producto
ORDER BY
  total_devoluciones DESC
LIMIT 5000;



**Tu Tarea:**

1.  **Analiza el Plan:** Ejecuta `EXPLAIN` sobre la consulta anterior. ¿Identificas algún problema? Fíjate bien si las tablas de hechos (`devoluciones_silver`, `pedidos_silver`) están siendo escaneadas por completo (sin `PartitionFilters`).
2.  **Reescribe la Consulta:** Modifica la consulta para aplicar las mejores prácticas. **Pista:** La solicitud de negocio contiene un filtro de fecha crucial (`primera semana de Febrero de 2023`). ¿Puedes añadirlo al `WHERE` para aprovechar el particionamiento de las tablas de hechos?
3.  **Verifica la Mejora:** Ejecuta `EXPLAIN` sobre tu nueva consulta optimizada. Compara el plan físico con el anterior. ¿Aparecen ahora los `PartitionFilters` en el escaneo de las tablas de hechos?